In [ ]:
# lotoのcsvの第・回、年月日を除く

import csv
from datetime import datetime

with open('loto7.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row['no'] = row['no'].replace('第','').replace('回','')
        row['date'] = datetime.strptime(row['date'], '%Y年%m月%d日').strftime('%Y-%m-%d')

In [ ]:
# lotoのcsvの第・回、年月日を除く

import csv
from datetime import datetime

# 読み込むCSVファイル
input_file = 'miniloto.csv'
# 書き出すCSVファイル（オリジナルを上書きする場合は 'miniloto.csv' に戻してください）
output_file = 'miniloto_processed.csv'

# データを読み込んで加工
with open(input_file, 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    # ヘッダーと加工済みデータを格納するためのリスト
    rows = []
    for row in reader:
        row['no'] = row['no'].replace('第', '').replace('回', '')
        row['date'] = datetime.strptime(row['date'], '%Y年%m月%d日').strftime('%Y-%m-%d')
        rows.append(row)  # 加工済みの行をリストに追加

# 加工済みデータを新しいCSVファイルに書き出し
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = rows[0].keys()  # ヘッダーの取得
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()  # ヘッダーの書き込み
    writer.writerows(rows)  # データの書き込み

print("CSVの加工が完了しました。")


In [ ]:
# 抜けているnoを探す

import csv

def find_missing_numbers(filename):
    # no カラムの値を格納するリスト
    no_list = []

    # CSV ファイルを読み込み、no カラムの値をリストに追加
    with open(filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            no_list.append(int(row['no']))

    # 開始と終了の番号範囲を取得
    min_no = min(no_list)
    max_no = max(no_list)

    # 全範囲の番号セットから取得した番号セットを差し引き
    missing_numbers = sorted(set(range(min_no, max_no + 1)) - set(no_list))

    return missing_numbers

# ファイル名を指定
filename = 'data.csv'
missing_nos = find_missing_numbers(filename)

print("Missing numbers:", missing_nos)

In [ ]:
import csv

def split_n34_csv(input_file, n3_file, n4_file):
    # `n34.csv` を読み込み、`n3.csv` と `n4.csv` を作成
    with open(input_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # `n3.csv` と `n4.csv` の出力ファイルを準備
        with open(n3_file, 'w', newline='', encoding='utf-8') as n3csv, open(n4_file, 'w', newline='', encoding='utf-8') as n4csv:
            # `n3.csv` のヘッダーを書き込み
            n3_writer = csv.DictWriter(n3csv, fieldnames=['no', 'date', 'n3'])
            n3_writer.writeheader()
            
            # `n4.csv` のヘッダーを書き込み
            n4_writer = csv.DictWriter(n4csv, fieldnames=['no', 'date', 'n4'])
            n4_writer.writeheader()

            # 各行に対して `n3.csv` と `n4.csv` に書き込み
            for row in reader:
                # `n3.csv` には `no`, `date`, `n3` 列のみ書き込み
                n3_writer.writerow({'no': row['no'], 'date': row['date'], 'n3': row['n3']})
                
                # `n4.csv` には `no`, `date`, `n4` 列のみ書き込み
                n4_writer.writerow({'no': row['no'], 'date': row['date'], 'n4': row['n4']})

# ファイル名を指定
input_file = 'output.csv'
n3_file = 'n3.csv'
n4_file = 'n4.csv'

# 分割実行
split_n34_csv(input_file, n3_file, n4_file)

print("n3.csv と n4.csv の作成が完了しました。")


In [2]:
#csvファイルからdbに情報を移す。登録する。

import csv
import sqlite3

# データベースに接続
conn = sqlite3.connect('lotodata.db')
cursor = conn.cursor()

# n3というテーブルが存在する場合に削除（前の回答で説明済み）
cursor.execute("DROP TABLE IF EXISTS n3;")

# n3というテーブルを作成
cursor.execute('''
    CREATE TABLE n3 (
        no INTEGER,
        date TEXT,
        main1 INTEGER,
        main2 INTEGER,
        main3 INTEGER,
        PRIMARY KEY (no)
    );
''')

# # CSVファイルからデータを読み込んで挿入
# with open('n3.csv', 'r', newline='') as csvfile:
#     reader = csv.DictReader(csvfile)
#     for row in reader:
#         cursor.execute('''
#             INSERT INTO n3 (no, date, main1, main2, main3)
#             VALUES (?, ?, ?, ?, ?);
#         ''', (row['no'], row['date'], row['main1'], row['main2'], row['main3']))

# CSVファイルからデータを読み込んで挿入
with open('n3.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # n3を個別の桁に分解
        n3 = f"{int(row['n3']):03}"  # 3桁にゼロパディング
        main1, main2, main3 = int(n3[0]), int(n3[1]), int(n3[2])

        # データを挿入
        cursor.execute('''
            INSERT INTO n3 (no, date, main1, main2, main3)
            VALUES (?, ?, ?, ?, ?);
        ''', (row['no'], row['date'], main1, main2, main3))

# n4というテーブルが存在する場合に削除
cursor.execute("DROP TABLE IF EXISTS n4;")

# n4というテーブルを作成
cursor.execute('''
    CREATE TABLE n4 (
        no INTEGER,
        date TEXT,
        main1 INTEGER,
        main2 INTEGER,
        main3 INTEGER,
        main4 INTEGER,
        PRIMARY KEY (no)
    );
''')

# # CSVファイルからデータを読み込んで挿入
# with open('n4.csv', 'r', newline='') as csvfile:
#     reader = csv.DictReader(csvfile)
#     for row in reader:
#         cursor.execute('''
#             INSERT INTO n4 (no, date, main1, main2, main3, main4)
#             VALUES (?, ?, ?, ?, ?, ?);
#         ''', (row['no'], row['date'], row['main1'], row['main2'], row['main3'], row['main4']))

# CSVファイルからデータを読み込んで挿入
with open('n4.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # n4を個別の桁に分解
        n4 = f"{int(row['n4']):04}"  # 3桁にゼロパディング
        main1, main2, main3,main4 = int(n4[0]), int(n4[1]), int(n4[2]), int(n4[3])

        # データを挿入
        cursor.execute('''
            INSERT INTO n4 (no, date, main1, main2, main3, main4)
            VALUES (?, ?, ?, ?, ?, ?);
        ''', (row['no'], row['date'], main1, main2, main3, main4))

# loto6というテーブルが存在する場合に削除
cursor.execute("DROP TABLE IF EXISTS loto6;")

# loto6というテーブルを作成
cursor.execute('''
    CREATE TABLE loto6 (
        no INTEGER,
        date TEXT,
        main1 INTEGER,
        main2 INTEGER,
        main3 INTEGER,
        main4 INTEGER,
        main5 INTEGER,
        main6 INTEGER,
        bonus INTERGER,
        PRIMARY KEY (no)
    );
''')

# CSVファイルからデータを読み込んで挿入
with open('loto6.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        cursor.execute('''
            INSERT INTO loto6 (no, date, main1, main2, main3, main4, main5, main6, bonus)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
        ''', (row['no'], row['date'], row['main1'], row['main2'], row['main3'], row['main4'], row['main5'], row['main6'], row['bonus']))

# loto7というテーブルが存在する場合に削除
cursor.execute("DROP TABLE IF EXISTS loto7;")

# loto7というテーブルを作成
cursor.execute('''
    CREATE TABLE loto7 (
        no INTEGER,
        date TEXT,
        main1 INTEGER,
        main2 INTEGER,
        main3 INTEGER,
        main4 INTEGER,
        main5 INTEGER,
        main6 INTEGER,
        main7 INTEGER,
        bonus1 INTEGER,
        bonus2 INTEGER,
        PRIMARY KEY (no)
    );
''')

# CSVファイルからデータを読み込んで挿入
with open('loto7.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        cursor.execute('''
            INSERT INTO loto7 (no, date, main1, main2, main3, main4, main5, main6, main7, bonus1, bonus2)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        ''', (row['no'], row['date'], row['main1'], row['main2'], row['main3'], row['main4'], row['main5'], row['main6'], row['main7'], row['bonus1'], row['bonus2']))

# minilotoというテーブルが存在する場合に削除
cursor.execute("DROP TABLE IF EXISTS miniloto;")

# minilotoというテーブルを作成
cursor.execute('''
    CREATE TABLE miniloto (
        no INTEGER,
        date TEXT,
        main1 INTEGER,
        main2 INTEGER,
        main3 INTEGER,
        main4 INTEGER,
        main5 INTEGER,
        bonus INTERGER,
        PRIMARY KEY (no)
    );
''')

# CSVファイルからデータを読み込んで挿入
with open('miniloto.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        cursor.execute('''
            INSERT INTO miniloto (no, date, main1, main2, main3, main4, main5, bonus)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?);
        ''', (row['no'], row['date'], row['main1'], row['main2'], row['main3'], row['main4'], row['main5'], row['bonus']))

# bingoというテーブルが存在する場合に削除
cursor.execute("DROP TABLE IF EXISTS bingo;")

# bingoというテーブルを作成
cursor.execute('''
    CREATE TABLE bingo (
        no INTEGER,
        date TEXT,
        main1 INTEGER,
        main2 INTEGER,
        main3 INTEGER,
        main4 INTEGER,
        main5 INTEGER,
        main6 INTEGER,
        main7 INTEGER,
        main8 INTEGER,
        PRIMARY KEY (no)
    );
''')

# CSVファイルからデータを読み込んで挿入
with open('bingo5.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        cursor.execute('''
            INSERT INTO bingo (no, date, main1, main2, main3, main4, main5, main6, main7, main8)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        ''', (row['no'], row['date'], row['main1'], row['main2'], row['main3'], row['main4'], row['main5'], row['main6'], row['main7'], row['main8']))

# qooというテーブルが存在する場合に削除
cursor.execute("DROP TABLE IF EXISTS qoo;")

# qooというテーブルを作成
cursor.execute('''
    CREATE TABLE qoo (
        no INTEGER,
        date TEXT,
        main1 TEXT,
        main2 TEXT,
        main3 TEXT,
        main4 TEXT,
        PRIMARY KEY (no)
    );
''')

# CSVファイルからデータを読み込んで挿入
with open('qoo.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        cursor.execute('''
            INSERT INTO qoo (no, date, main1, main2, main3, main4)
            VALUES (?, ?, ?, ?, ?, ?);
        ''', (row['no'], row['date'], row['main1'], row['main2'], row['main3'], row['main4']))

# 変更を保存
conn.commit()

# 接続を閉じる
conn.close()

In [ ]:
# ナンバーズのゾロ目検索

import sqlite3

# データベースに再接続
conn = sqlite3.connect('lotodata.db')
cursor = conn.cursor()

# n3 のゾロ目の日付を取得
cursor.execute('''
    SELECT date, main1, main2, main3
    FROM n3
    WHERE main1 = main2 AND main2 = main3;
''')
n3_zeros = cursor.fetchall()

print(f"n3 のゾロ目の日付(合計:{len(n3_zeros)}):")
for row in n3_zeros:
    print(f"日付: {row[0]}, 数値: {row[1]}{row[2]}{row[3]}")

# n4 のゾロ目の日付を取得
cursor.execute('''
    SELECT date, main1, main2, main3, main4
    FROM n4
    WHERE main1 = main2 AND main2 = main3 AND main3 = main4;
''')
n4_zeros = cursor.fetchall()

print(f"\nn4 のゾロ目の日付(合計:{len(n4_zeros)}:")
for row in n4_zeros:
    print(f"日付: {row[0]}, 数値: {row[1]}{row[2]}{row[3]}{row[4]}")

# 接続を閉じる
conn.close()


In [ ]:
# くじデータベースのチェック用
import sqlite3

date = '2024-12-03'

# 日付でアップデートするデータをまとめて、jsonにする関数
def all_loto_result_searched_by_date(date):
    conn = sqlite3.connect('lotodata.db')
    c = conn.cursor()
    lotos = ["qoo", "n3", "n4", "bingo", "miniloto", "loto6", "loto7"]
    result_dict = {}
    for loto in lotos:
        # プレースホルダーを使用してSQLクエリを安全に構築
        c.execute(f"SELECT * FROM {loto} WHERE date > ?", (date,))
        rows = c.fetchall()
        header = [description[0] for description in c.description]
        result = [{header[i]: value for i, value in enumerate(row)} for row in rows]
        result_dict[loto]=result
    conn.commit()
    conn.close()
    return result_dict

result = all_loto_result_searched_by_date(date=date)
print(result)

In [ ]:
# くじアプリローカルテスト用スクリプト
# /get_loto_results?pw=doraemon810&date=2024-02-02

from flask import Flask, request, jsonify
import sqlite3
from datetime import datetime

app = Flask(__name__)
# app.json.ensure_ascii = False
app.config['JSON_AS_ASCII'] = False


valid_password = 'doraemon810'

@app.before_request
def check_authentication():
    pw = request.args.get('pw')
    if pw != valid_password:
        return jsonify({'error': 'c'}), 401

# テスト用loto6の結果のみ
@app.route('/test')
def test_get_loto_results():
    date = request.args.get('date')

    if date is None:
        return jsonify({'error': 'd'}), 400

    # 日付が正しい形式か確認
    if not is_valid_date(date):
        return jsonify({'error': 'vd'}), 400

    # 認証が成功した場合のみデータベースに接続
    conn = sqlite3.connect('lotodata.db')
    c = conn.cursor()

    lotos = ["loto6"]
    result_dict = {}
    for loto in lotos:
        c.execute(f"SELECT * FROM {loto} WHERE date > ?", (date,))
        rows = c.fetchall()
        header = [description[0] for description in c.description]
        result = [{header[i]: value for i, value in enumerate(row)} for row in rows]
        result_dict[loto] = result

    conn.close()
    return jsonify(result_dict)

@app.route('/get_loto_results')
def get_loto_results():
    date = request.args.get('date')

    if date is None:
        return jsonify({'error': 'd'}), 400

    # 日付が正しい形式か確認
    if not is_valid_date(date):
        return jsonify({'error': 'vd'}), 400

    # 認証が成功した場合のみデータベースに接続
    conn = sqlite3.connect('lotodata.db')
    c = conn.cursor()

    lotos = ["qoo", "n3", "n4", "bingo", "miniloto", "loto6", "loto7"]
    result_dict = {}
    for loto in lotos:
        c.execute(f"SELECT * FROM {loto} WHERE date > ?", (date,))
        rows = c.fetchall()
        header = [description[0] for description in c.description]
        result = [{header[i]: value for i, value in enumerate(row)} for row in rows]
        result_dict[loto] = result

    conn.close()
    return jsonify(result_dict)

def is_valid_date(date):
    try:
        # 日付文字列をdatetimeオブジェクトに変換し、エラーが発生しなければ有効な日付と見なす
        datetime.strptime(date, '%Y-%m-%d')
        return True
    except ValueError:
        # エラーが発生した場合は無効な日付と見なす
        return False

if __name__ == '__main__':
    app.run()
